# <center>Лабораторна робота №3
## <center>Дерева рішень в іграшкової завданню і на даних Adult сховища UCI

    
**У лабораторній роботі Вам пропонується розібратися з тим, як працює дерево рішень, на іграшковому прикладі, потім навчити і налаштувати дерева і (при бажанні) випадковий ліс в завданні класифікації на даних Adult сховища UCI. Дайте відповідь на всі питання в цій зошиті.**

Підключаємо необхідні бібліотеки

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Частина 1. Іграшковий набір даних "дівчина в барі"

**Мета - "на пальцях", за допомогою іграшкової завдання класифікації розібратися в тому, як працюють дерева рішень. Саме по собі дерево рішень - досить слабкий алгоритм, але засновані на ньому алгоритми випадкового лісу і градиентного бустінга - мабуть, найкраще, що є на сьогоднішній день (в задачах, де можна обійтися без нейронних мереж). Тому розібратися в тому, як працює дерево рішень, корисно.**

**Розглянемо іграшкову завдання бінарної класифікації: чи поїде з Вами дівчина з бару? Це буде залежати від Вашої зовнішності і красномовства, фортеці пропонованих напоїв і, як це не меркантильно, від кількості витрачених в барі грошей.**

### Створення набору даних

In [2]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ['Внешность', 'Алкоголь_в_напитке',
            'Уровень_красноречия', 'Потраченные_деньги']

**Навчальна вибірка**

In [4]:
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая',
                         'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий',
                                   'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много',
                                  'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,0,1,1,0,1,0,0,0,1
1,1,0,1,1,0,0,1,0,1,0
2,0,0,1,0,1,0,0,1,0,1
3,1,1,0,0,1,0,0,1,1,0
4,1,1,0,1,0,0,1,0,0,1
5,0,1,0,1,0,1,0,0,0,1
6,0,0,1,1,0,0,0,1,0,1


**Тестова вибірка**

In [5]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,1,0,1,0,1,0,1
1,0,1,1,0,1,0,1,0
2,1,0,1,0,0,1,0,1


In [6]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train['Поедет']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Внешность_отталкивающая,Уровень_красноречия_высокий,Внешность_приятная,Потраченные_деньги_много,Алкоголь_в_напитке_нет,Уровень_красноречия_средний,Алкоголь_в_напитке_да,Потраченные_деньги_мало
0,0,1,1,1,0,0,1,0
1,0,0,1,0,0,0,1,1
2,0,0,1,1,1,1,0,0
3,1,0,0,0,1,1,0,1
4,1,0,0,1,0,0,1,0
5,1,1,0,1,0,0,1,0
6,0,0,1,1,0,1,1,0


In [7]:
df_test

,Потраченные_деньги_мало,Алкоголь_в_напитке_нет,Потраченные_деньги_много,Уровень_красноречия_высокий,Алкоголь_в_напитке_да,Уровень_красноречия_средний,Внешность_приятная,Внешность_отталкивающая
0,0,1,1,0,0,1,1,0
1,1,0,0,1,1,0,1,0
2,0,0,1,0,1,1,0,1


**Побудуйте від руки (або в графічному редакторі) дерево рішень для цього набору даних. Додатково (для бажаючих) - можете зробити отрисовку дерева і написати код для побудови всього дерева.**

<font color = 'red'>Питання 1. </font> Яка ентропія початкової системи ($S_0$)? Під станами системи розуміємо значення ознаки "Поїде" - 0 або 1 (тобто всього 2 стану).

<font color = 'red'> Питання 2. </font> Розглянемо розбиття навчальної вибірки за ознакою "Зовнішність \ _пріятная". Яка ентропія $S_1$ лівої групи, тих, у кого зовнішність приємна, і правої групи - $S_2$? Який приріст інформації при даному розбитті (IG)?

**Побудуйте за допомогою `sklearn` дерево рішень, навчивши його на навчальній вибірці. Глибину можна не обмежувати.**

In [9]:
# Ваш код здесь

**Додатково: відобразите дерево за допомогою graphviz. Можна використовувати `pydot` або, наприклад, [онлайн-сервіс](https://www.coolutils.com/ru/online/DOT-to-PNG) dot2png.**

In [10]:
# Ваш код здесь

## Частина 2. Функції для розрахунку ентропії і приросту інформації

Приклад для перевірки: 9 синіх кульок і 11 жовтих. Нехай кулька знаходиться в стані "1", якщо він синій і "0" - якщо він жовтий.

In [11]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

![png](../img/decision_tree3.png)

Далі нехай кульки розбиваються на 2 групи
![png](../img/decision_tree4.png)

In [12]:
# две группы
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых

**Реалізуйте функцію для розрахунку ентропії Шеннона.**

In [13]:
def entropy(a_list):
    
    # Ваш код здесь
    pass

Перевірка

In [ ]:
print(entropy(balls)) # 9 синих и 11 желтых
print(entropy(balls_left)) # 8 синих и 5 желтых
print(entropy(balls_right)) # 1 синий и 6 желтых

<font color = 'red'>Питання 3.</font> Чому дорівнює ентропія стану, заданого списком `balls_left`?

<font color='red'>Питання 4.</font> Чому дорівнює ентропія гральної кістки з незміщеної центром тяжіння?

In [15]:
# расчет прироста информации

def information_gain(root, left, right):
    ''' root - изначальный набор данных, left и right два разбиения изначального набора'''
    
    # Ваш код здесь
    pass

<font color='red'>Питання 5.  </font> Який приріст інформації при поділі вибірки на `balls_left` і` balls_right`?

In [16]:
# Ваш код здесь

In [17]:
def best_feature_to_split(X, y):
    ''' Выводит прирост информации при разбиении по каждому признаку'''
    
    # Ваш код здесь
    pass

## Частина 3. Дерево рішень без налаштування параметрів

**Навчіть на наявної вибірці дерево рішень ( `DecisionTreeClassifier`) максимальної глибини 3 і отримаєте якість на тесті. Використовуйте параметр `random_state` = 17 для відтворюваності результатів.**

In [ ]:
tree = # Ваш код здесь
tree.fit # Ваш код здесь

**Зробіть за допомогою отриманої моделі прогноз для тестової вибірки.**

In [ ]:
tree_predictions = tree.predict # Ваш код здесь

In [ ]:
accuracy_score # Ваш код здесь

<font color='red'>Питання 6.  </font> Яка частка правильних відповідей дерева рішень на тестовій вибірці при максимальній глибині дерева = 3 і random_state = 17?

## 3.2. Дерево рішень з налаштуванням параметрів

**Навчіть на наявної вибірці дерево рішень ( `DecisionTreeClassifier`, знову` random_state` = 17). Максимальну глибину налаштуйте на крос-валідації за допомогою `GridSearchCV`. Проведіть 5-кратну крос-валідацію.**

In [ ]:
tree_params = {'max_depth': range(2,11)}

locally_best_tree = GridSearchCV # Ваш код здесь                      

locally_best_tree.fit # Ваш код здесь

In [ ]:
print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

**Навчіть на наявної вибірці дерево рішень максимальної глибини 9 (це краще значення `max_depth` в цьому випадку) та оцініть частку правильних відповідей на тесті. Використовуйте параметр random_state = 17 для відтворюваності результатів.**

In [ ]:
tuned_tree = # Ваш код здесь
tuned_tree.fit # Ваш код здесь
tuned_tree_predictions = tuned_tree.predict # Ваш код здесь
accuracy_score # Ваш код здесь

<font color='red'>Питання 7.  </font> Яка частка правильних відповідей дерева рішень на тестовій вибірці при максимальній глибині дерева = 9 і random_state = 17?